# Import OpenTelemetry Modules for Traces

In [ ]:
from opentelemetry import baggage, trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import (
    ConsoleSpanExporter,
    BatchSpanProcessor
)
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.trace import Status, StatusCode
import socket

# Create a TraceProvider that we can send trace spans to

In [ ]:
provider = TracerProvider(resource=Resource.create({
    # "service.name": "section01",
    "service.name": __name__,
    # "datadog.host.use_as_metadata": True,
    "deployment.environment": "otel",
    "host.name": socket.gethostname(),
}))
print(provider.__dict__)

# Create a new trace span and send it to the collector

In [ ]:
# provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))
provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint="localhost:4317", insecure=True)))
tracer = trace.get_tracer("python", tracer_provider=provider)

In [ ]:
with tracer.start_as_current_span("parent_span") as parent_span:
    ctx = baggage.set_baggage("foo", "bar")

    parent_span.set_status(Status(StatusCode.OK))
    # do some work that 'parent' tracks
    print("doing some work...")

    # Create a nested span to track nested work
    with tracer.start_as_current_span("child") as child:
        # do some work that 'child' tracks
        print("doing some nested work...")
        # the nested span is closed when it's out of scope

trace_id = trace.format_trace_id(parent_span.context.trace_id)
print(f"Global context baggage: {baggage.get_all()}")
print(f"Span context baggage: {baggage.get_all(context=ctx)}")
print(f"Context: {parent_span.context}")
print(f"Trace ID: {trace_id}")
print(f"View trace in Datadog: https://app.datadoghq.com/apm/traces?query=%40otel.trace_id%3A{trace_id}")